In [1]:
import datetime

def generate_csv_filename(base_name):
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%d%m%Y_%H%M%S_%m%S")
    file_extension = ".csv"
    return f"{base_name}_{formatted_datetime}{file_extension}"

# Exemple d'utilisation :
file_name = generate_csv_filename("toto")
print(file_name)


toto_11102023_180402_1002.csv


In [3]:
import os
import requests
import pandas as pd

# Fonction pour envoyer une requête à l'API avec des paramètres de filtrage
def send_api_request(filter_value):
    try:
        api_url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"
        headers = {
            'X-RapidAPI-Key': "82e7f76641msh8efe42731f3e34bp1164ddjsn1d61cc9d0dc1",
            'X-RapidAPI-Host': "edamam-food-and-grocery-database.p.rapidapi.com"
        }
#         params = {
#             "nutrition-type": "cooking",
#             "category[0]": "fast-food",  # Changer la catégorie "category[0]": "generic-foods",
#             "health[0]": "alcohol-free",
#             "q": filter_value
#         }
        params = {
            "ingr": "champagne",
            "nutrition-type": "cooking",
            "category[0]": "generic-foods",
            "health[0]": "alcohol-free"
        }
#         params = {
#             "from": 0,  # Obtenir les premiers éléments
#             "to": 10  # Obtenir les 50 premiers éléments
#         }

        response = requests.get(api_url, headers=headers, params=params)

        if response.status_code != 200:
            print("Erreur lors de la requête à l'API.")
            return None

        api_data = response.json()
        hints = api_data.get("hints", [])

        if not hints:
            print("Aucune donnée trouvée pour le filtre spécifié.")
            return None

        return hints
    except Exception as e:
        print(f"Erreur lors de l'envoi de la requête API : {str(e)}")
        return None

# Fonction pour filtrer les données et les sélectionner
def filter_and_select_data(api_data, selected_columns):
    filtered_data = []
    for hint in api_data:
        food = hint.get("food", {})
        filtered_item = {col: food.get(col) for col in selected_columns}
        filtered_data.append(filtered_item)
    return filtered_data

# Fonction pour enregistrer les données filtrées dans un fichier CSV
def save_filtered_data_to_csv(filtered_data, output_dir, output_csv_file):
    try:
        df = pd.DataFrame(filtered_data)
        output_file_path = os.path.join(output_dir, output_csv_file)
        df.to_csv(output_file_path, index=False)
        print(f"Données filtrées enregistrées avec succès dans {output_file_path}")
    except Exception as e:
        print(f"Erreur lors de l'enregistrement des données filtrées dans un fichier CSV : {str(e)}")

# Fonction principale
def main():
    output_dir = r'C:\Users\Zbook\OpenClassRoom\ProjetsGIT\Projet_6\data'
    output_csv_file =generate_csv_filename("donnees_filtrees_vf_")
    filter_value = "champagne"
    selected_columns = ["foodId", "label", "category", "categoryLabel", "knownAs", "image"]

    api_data = send_api_request(filter_value)

    if api_data is not None:
        filtered_data = filter_and_select_data(api_data, selected_columns)
        save_filtered_data_to_csv(filtered_data, output_dir, output_csv_file)

if __name__ == "__main__":
    main()


Données filtrées enregistrées avec succès dans C:\Users\Zbook\OpenClassRoom\ProjetsGIT\Projet_6\data\donnees_filtrees_vf__11102023_180508_1008.csv


In [12]:
import os
import requests
import pandas as pd

def filter_and_save_api_data(output_dir, output_csv_file, filter_column, filter_value, selected_columns):
    try:
        # Paramètres de la requête API avec les critères de filtrage
        api_url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"
        headers = {
            'X-RapidAPI-Key': "82e7f76641msh8efe42731f3e34bp1164ddjsn1d61cc9d0dc1",
            'X-RapidAPI-Host': "edamam-food-and-grocery-database.p.rapidapi.com"
        }
#         params = {
#             "nutrition-type": "cooking",
#             "category[0]": "fast-food" , #"generic-foods",
#             "health[0]": "alcohol-free",
#             "q": filter_value  # Filtrer par la valeur spécifiée
#         }
        params = {
            "ingr": "champagne",
            "nutrition-type": "cooking",
            "category[0]": "generic-foods",
            "health[0]": "alcohol-free"
        }
        # Envoyer la requête à l'API
        print("api_url " , api_url)
        print("headers " , headers)
        print("params " , params)
        response = requests.get(api_url, headers=headers, params=params)

        if response.status_code != 200:
            print("Erreur lors de la requête à l'API.")
            return

        # Convertir la réponse JSON en DataFrame
        api_data = response.json()
        hints = api_data.get("hints", [])

        if not hints:
            print("Aucune donnée trouvée pour le filtre spécifié.")
            return

        # Créer une liste des données filtrées avec les colonnes sélectionnées
        filtered_data = []
        for hint in hints:
            food = hint.get("food", {})
            filtered_item = {col: food.get(col) for col in selected_columns}
            filtered_data.append(filtered_item)

        # Créer un DataFrame à partir des données filtrées
        df = pd.DataFrame(filtered_data)

        # Créer le chemin complet du fichier de sortie
        output_file_path = os.path.join(output_dir, output_csv_file)

        # Enregistrer les données filtrées dans un fichier CSV
        df.to_csv(output_file_path, index=False)

        print(f"Données filtrées enregistrées avec succès dans {output_file_path}")
    except Exception as e:
        print(f"Erreur lors de la filtration et de l'enregistrement CSV : {str(e)}")

# Exemple d'utilisation :
output_dir = r'C:\Users\Zbook\OpenClassRoom\ProjetsGIT\Projet_6\data'  # Chemin du répertoire de sauvegarde
output_csv_file =generate_csv_filename("donnees_filtrees_vfc_") # Nom du fichier de sortie
filter_column = "label"  # Colonne à filtrer (peut varier en fonction de la structure de la réponse de l'API)
filter_value = "champagne"  # Valeur du filtre
selected_columns = ["foodId", "label", "category", "categoryLabel", "knownAs","image"]  # Colonnes sélectionnées

filter_and_save_api_data(output_dir, output_csv_file, filter_column, filter_value, selected_columns)


api_url  https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser
headers  {'X-RapidAPI-Key': '82e7f76641msh8efe42731f3e34bp1164ddjsn1d61cc9d0dc1', 'X-RapidAPI-Host': 'edamam-food-and-grocery-database.p.rapidapi.com'}
params  {'ingr': 'champagne', 'nutrition-type': 'cooking', 'category[0]': 'generic-foods', 'health[0]': 'alcohol-free'}
Données filtrées enregistrées avec succès dans C:\Users\Zbook\OpenClassRoom\ProjetsGIT\Projet_6\data\donnees_filtrees_vfc__04102023_234612_1012.csv
